In [1]:
from abc import ABC, abstractmethod
import pandas as pd
from typing import Literal

class AbstractCalculoDataframes(ABC):
    
    @abstractmethod
    def convertir_a_euros(self, main):
        pass
    
    @abstractmethod
    def unir_dataframes_por_columnas_comunes(self, df1, df2, how: Literal['left', 'right', 'outer', 'inner'] = 'inner'):
        return pd.DataFrame()
    
class CalculoDataframes(AbstractCalculoDataframes):
    
    def convertir_a_euros(self, main):
        # Definir las tasas de cambio
        trm_marzo_2024 = 4253.92
        trm_abril_2024 = 4226.47
        trm_promedio_2024 = 4652.35

        # Crear una copia del DataFrame para evitar el SettingWithCopyWarning
        main = main.copy()

        # Convertir los valores de 'Real' y 'Presupuesto' a euros
        main['RealEuros'] = main['Real'] / trm_abril_2024
        main['PresupuestoEuros'] = main['Presupuesto'] / trm_promedio_2024
        main['PorcentajeCumplimientoEuros'] = (main['RealEuros']/main['PresupuestoEuros']).apply(lambda x: round(x, 2))

        # Renombrar las columnas originales 'Real' y 'Presupuesto'
        main = main.rename(columns={
            'Real': 'RealCOP', 'Presupuesto': 'PresupuestoCOP', 
            'PorcentajeCumplimiento': 'PorcentajeCumplimientoCOP',
            'PorcentajeCumplimientoEuros': 'PorcentajeCumplimiento'
        })

        # Renombrar las nuevas columnas para reflejar que están en euros
        main = main.rename(columns={'RealEuros': 'Real', 'PresupuestoEuros': 'Presupuesto'})
        return main[
            ['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento',
            'RealCOP', 'PresupuestoCOP', 'PorcentajeCumplimientoCOP']
        ]

    def convertir_valores_a_euros(self, dataframe, variables_convertir_euros):
        """
        Convierte los valores de las variables especificadas a euros en el DataFrame dado.
        
        Parámetros:
        - dataframe: DataFrame al que se le aplicará la conversión.
        - variables_convertir_euros: Lista de nombres de variables cuyos valores se convertirán a euros.
        
        Retorna:
        - DataFrame con los valores convertidos a euros.
        """
        # Filtrar el DataFrame original para las variables que se van a convertir
        dataframe_aplicar_euros = dataframe[dataframe['Variable'].isin(variables_convertir_euros)]
        
        # Hacer una copia del DataFrame original para evitar cambios inesperados
        dataframe_convertido_euros = dataframe.copy()
        
        # Inicializar columnas de valores en euros
        dataframe_convertido_euros['RealCOP'] = 0
        dataframe_convertido_euros['PresupuestoCOP'] = 0
        dataframe_convertido_euros['PorcentajeCumplimientoCOP'] = 0
        
        # Actualizar las columnas de valores en euros con los valores convertidos
        dataframe_convertido_euros.update(self.convertir_a_euros(dataframe_aplicar_euros))
        
        return dataframe_convertido_euros

    def unir_dataframes_por_columnas_comunes(self, df1, df2, how='inner'):
        """
        Une dos DataFrames basándose en sus columnas comunes con un tipo de unión especificado.
        
        Parámetros:
        - df1: Primer DataFrame.
        - df2: Segundo DataFrame.
        - how: Tipo de unión ('left', 'right', 'outer', 'inner'). Por defecto es 'inner'.
        
        Retorna:
        - DataFrame resultante de la unión.
        """
        # Encuentra las columnas comunes entre ambos DataFrames
        columnas_comunes = df1.columns.intersection(df2.columns)
        print(columnas_comunes)
        # Une los DataFrames basándose en las columnas comunes y el tipo de unión especificado
        df_unido = pd.merge(df1, df2, on=columnas_comunes.tolist(), how=how)
        
        return df_unido